In [7]:
import numpy as np
from collections import Counter
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [9]:
df = pd.read_csv('final_data2.csv')

In [11]:
flair_dict = {   
 0: 'News',
 1: 'FUNDAMENTALS',
 2: 'NEWS',
 3: 'DISCUSSION',
 4: 'TECHNICALS',
 5: 'Comedy',
 6: 'COMEDY',
 7: 'Exchange',
 8: 'SENTIMENT',
 9: 'STRATEGY',
 10: 'Strategy',
 11: 'Technical Analysis',
 12: 'Educational'
}

In [10]:
df.head()

,flair,title,url,text,score,num_comments,original,comments,polarity
0,0,Ethereum Fees Now at July 2021 Lows of $2.54 -...,https://cryptonewsland.com/ethereum-fees-now-a...,NaN,21,14,False,Ethereum [pros](/r/CryptoMarkets/comments/ux7...,0.9497
1,0,Coinbase CEO’s Response to Employee Rant Backf...,https://cryptonewsland.com/coinbase-ceos-respo...,NaN,4,7,False,USDC [pros](/r/CryptoMarkets/comments/v9izys/...,-0.4404
2,0,Mars4 Metaverse is Selling Fast in Japan – Spo...,https://news.bitcoin.com/mars4-metaverse-is-se...,NaN,2,9,False,Bitcoin [pros](/r/CryptoMarkets/comments/v9vc...,-0.2732
3,0,Terra Crisis: G-7 Nations Call for Urgent Cryp...,https://crypto.news/terra-g-7-nations-urgent-c...,NaN,14,13,False,Do Kwan screwed us. They were just looking fo...,0.6705
4,0,White House says the Administration is Closely...,https://www.youtube.com/watch?v=4oSL1gCHFdQ,NaN,0,4,False,Inflation [pros](/r/CryptoMarkets/comments/vd...,0.8979


In [34]:
def print_rand_example(df, col_name, col_value, chars=180):    
    title = df[df[col_name]==col_value].sample()['title']
    url = df[df[col_name]==col_value].sample()['url']
    text = df[df[col_name]==col_value].sample()['text']
    comments = df[df[col_name]==col_value].sample()['comments']
    print(title[0:chars] + "...")
    print(url[0:chars] + "...")
    print(text[0:chars] + "...")
    print(comments[0:chars] + "...")


In [37]:
for flair_num in range(13):
    print(f"Flair: {flair_dict[flair_num]}")
    print_rand_example(df, 'flair', flair_num)
    print()


Flair: News
86    Ukrainian President Volodymyr Zelenskyy legali...
Name: title, dtype: object
92    https://youtu.be/ycJW54ItBHM...
Name: url, dtype: object
74    The price of HBAR could reach $0.81 in 2022, a...
Name: text, dtype: object
26     NFT [pros](/r/CryptoMarkets/comments/udlknm/n...
Name: comments, dtype: object

Flair: FUNDAMENTALS
175    The majority of top crypto projects in the mar...
Name: title, dtype: object
184    https://youtu.be/03xxDEsmWv8...
Name: url, dtype: object
147    **Preface and Disclosures:**\n\nNone of the in...
Name: text, dtype: object
116     Monero [pros](/r/CryptoMarkets/comments/ui2qj...
Name: comments, dtype: object

Flair: NEWS
279    Proof-of-Work Limit is Back in Discussion at t...
Name: title, dtype: object
203    https://crypto.news/terra-g-7-nations-urgent-c...
Name: url, dtype: object
279    NaN
Name: text, dtype: object
248    NaN
Name: comments, dtype: object

Flair: DISCUSSION
383    Weekly Discussion Megathread - May 16, 2021 (G...
Na

In [53]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

num_classes = 13

y = tf.keras.utils.to_categorical(df["flair"].values, num_classes=num_classes)

x_train, x_test, y_train, y_test = train_test_split(df['title'], y, test_size=0.25)

# Loading pretrained BERT  

In [39]:
import tensorflow_hub as hub
import tensorflow_text as text

preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")


def get_embeddings(sentences):
    preprocessed_text = preprocessor(sentences)
    return encoder(preprocessed_text)['pooled_output']


get_embeddings([
    "Questa collezione di Haiku è una porta aperta sulla cultura giapponese."]
)

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[ 1.08269252e-01, -9.16300267e-02, -3.19653004e-01,
        -1.70694273e-02,  3.34133953e-01,  2.00266182e-01,
        -2.46936023e-01, -5.43311000e-01, -3.56956989e-01,
        -2.97796547e-01, -2.08980605e-01,  1.42025888e-01,
         2.72499211e-02, -4.33980763e-01, -5.73600113e-01,
        -8.82729530e-01,  2.37143829e-01, -1.87127650e-01,
        -2.56524444e-01, -2.46629804e-01,  6.32454097e-01,
         1.21354170e-01, -3.95606160e-01,  1.58425853e-01,
        -6.60168827e-02, -1.00859690e+00,  3.03424567e-01,
         2.40532998e-02, -9.84722450e-02, -6.36505425e-01,
         1.15401782e-01, -7.62528837e-01, -5.43886542e-01,
        -4.67391938e-01, -6.24043405e-01,  2.63592869e-01,
        -4.41221267e-01,  6.62042052e-02, -3.88788015e-01,
         1.61611661e-02,  1.15522750e-01, -4.35086757e-01,
        -5.12637496e-01,  2.74716794e-01, -3.44001085e-01,
        -1.88089877e-01,  1.52756153e-02, -3.23565960e-01,
      

In [43]:
#Computing cosine similarity
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

def plot_similarity(features, labels):
    cos_sim = cosine_similarity(features)
    fig = plt.figure(figsize=(10,8))
    sns.set(font_scale=1.2)
    cbar_kws=dict(use_gridspec=False, location="left")
    g = sns.heatmap(cos_sim, xticklabels=labels, yticklabels=labels,
      vmin=0, vmax=1, annot=True, cmap="Blues", cbar_kws=cbar_kws)
    g.tick_params(labelright=True, labelleft=False)
    g.set_yticklabels(labels, rotation=0)
    g.set_title("Semantic Textual Similarity")

In [45]:
#Precision, Recall, F1 Score
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Training the model

In [49]:
#Defining the model
import tensorflow as tf
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = preprocessor(i)
x = encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(13, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [73]:
my_acc = tf.keras.metrics.CategoricalAccuracy(name="accuracy")

In [56]:
# Training the model

n_epochs = 20

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
      balanced_recall,
      balanced_precision,
      balanced_f1_score
]

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", 
                                                      patience = 3,
                                                      restore_best_weights = True)

model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = METRICS)

model_fit = model.fit(x_train, 
                      y_train, 
                      epochs = n_epochs,
                      validation_data = (x_test, y_test),
                      callbacks = [earlystop_callback])

Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
27/27 [==============================] - ETA: 0s - loss: 2.2927 - accuracy: 0.2155 - balanced_recall: 0.0055 - balanced_precision: 0.0107 - balanced_f1_score: 0.0071WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x0000019C62419B88> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
27/27 [==============================] - 267s 10s/step - loss: 2.2927 - accuracy: 0.2155 - balanced_recall: 0.0055 - balanced_precision: 0.0107 - balanced_f1_score: 0.0071 - val_loss: 2.1149 - val_accuracy: 0.3000 - val_balanced_recall: 0.0164 - val_balanced_precision: 0.0342 - val_balanced_f1_score: 0.0205
Epoch 2/20
27/27 [==============================] - 224s 8s/step - loss: 1.9825 - accuracy: 0.3702 - balanced_recall: 0.0209 - balanced_precision: 0.0313 - balanced_f1_score: 0.0237 - val_loss: 1.9429 - val_accuracy: 0.3964 - val_balanced_recall: 0.0564 - val_balanced_precision: 0.0627 - val_balanced_f1_score: 0.0549
Epoch 3/20
27/27 [==============================] - 226s 8s/s

In [ ]:
model.save('/saved_model/my_model')


In [68]:
from joblib import Parallel, delayed
import joblib

joblib.dump(model, 'savedModel1.pkl')
 

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
INFO:tensorflow:Assets written to: ram://13fd048d-c666-4aab-8d58-ab5aad56ac14/assets


INFO:tensorflow:Assets written to: ram://13fd048d-c666-4aab-8d58-ab5aad56ac14/assets


['savedModel1.pkl']

# Results

In [89]:
# Loading the saved model
from tensorflow import keras

new_model = keras.models.load_model("/saved_model/my_model", compile=False)

ResourceExhaustedError: OOM when allocating tensor with shape[501153,768] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:RestoreV2]

In [110]:
# Using these titles from cryptomarkets subreddit to predit the flair
titles = ['Yahoo is Launching Metaverse Events in Hong Kong',
         "My prediction. BTC around 17K by Sunday. ETH around 700",
          "The markets will rebound, they always rebound for everything else, crypto shouldn't be the exception",
          "Through our actions we have turned crypto into just another financial asset and it had the potential for much more",
          "Federal Reserve announces Highest Interest Rate-hike since 1994"
         ]

In [111]:
res = [np.argmax(pred) for pred in model.predict(titles)]

for i in res:
    print(i, ' ', flair_dict[i])

1/1 [==============================] - 1s 1s/step
12   Educational
0   News
1   FUNDAMENTALS
12   Educational
2   NEWS
